
# Unit 2 — Team Classification (Flights, BQML)

**Goal (team):** Build an *ops-ready* classifier in **BigQuery ML** to predict **`diverted`** on U.S. flights. Minimal handholding by design.

**What you deliver (inside this notebook):**
- One **LOGISTIC_REG** model (baseline), one **engineered** model using `TRANSFORM`
- **Evaluation** via `ML.EVALUATE` and **confusion matrices** (default 0.5 + your custom threshold)
- **Threshold choice** + 3–5 sentence ops justification
- Embedded **rubric** below (self-check before submission)

> Choose *one* dataset table that exists at your institution:  
> • `bigquery-public-data.faa.us_flights` **or** `bigquery-public-data.flights.*`  
> Make sure the table has `carrier`, `dep_delay`, `arr_delay` (for filters), `origin`, `dest`, `diverted` (or equivalent).


**Note:** Since both the flights datasets have been removed from BigQuery, I used another flights dataset from Kaggle that is the most similar to the BigQuery flights datasets and contains the required feautures for regression analysis. Here is the link to the flights dataset from Kaggle: https://www.kaggle.com/datasets/usdot/flight-delays?resource=download&select=flights.csv

In [13]:

# --- Minimal setup (edit 3 vars) ---
from google.colab import auth
auth.authenticate_user()

import os
from google.cloud import bigquery

PROJECT_ID = "heroic-trilogy-471119-k8"      # e.g., mgmt-467-47888
REGION     = "us-central1"
TABLE_PATH = "heroic-trilogy-471119-k8.assignment2.flights"   # or your `bigquery-public-data.flights` table/view

os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["REGION"]     = REGION
bq = bigquery.Client(project=PROJECT_ID)

print("BQ Project:", PROJECT_ID)
print("Source table:", TABLE_PATH)


BQ Project: heroic-trilogy-471119-k8
Source table: heroic-trilogy-471119-k8.assignment2.flights


### Quick sanity check

In [15]:

preview_sql = f"SELECT * FROM `{TABLE_PATH}` LIMIT 5"
bq.query(preview_sql).result().to_dataframe()


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,10,29,4,AS,64,N762AS,14256,15841,1523,...,1547,2,0,0,None,<NA>,<NA>,<NA>,<NA>,<NA>
1,2015,10,6,2,AS,65,N779AS,15841,14256,1110,...,1125,-9,0,0,None,<NA>,<NA>,<NA>,<NA>,<NA>
2,2015,11,11,3,AS,65,N756AS,WRG,PSG,1106,...,1139,10,0,0,None,<NA>,<NA>,<NA>,<NA>,<NA>
3,2015,5,30,6,AS,64,N765AS,PSG,WRG,1530,...,1603,10,0,0,None,<NA>,<NA>,<NA>,<NA>,<NA>
4,2015,1,18,7,AS,65,N796AS,WRG,PSG,1100,...,1103,-23,0,0,None,<NA>,<NA>,<NA>,<NA>,<NA>



## 1) Canonical mapping (adjust as needed)
Map to a minimal schema used in the rest of the notebook:
- `flight_date` (DATE), `dep_delay` (NUM), `distance` (NUM), `carrier` (STRING), `origin` (STRING), `dest` (STRING), `diverted` (BOOL)


In [26]:
CANONICAL_BASE_SQL = f'''
WITH canonical_flights AS (
  SELECT
    PARSE_DATE('%Y-%m-%d', CONCAT(CAST(YEAR AS STRING), '-', CAST(MONTH AS STRING), '-', CAST(DAY AS STRING))) AS flight_date,
    CAST(DEPARTURE_DELAY AS FLOAT64) AS dep_delay,
    CAST(distance  AS FLOAT64) AS distance,
    CAST(AIRLINE   AS STRING)  AS carrier,
    CAST(ORIGIN_AIRPORT AS STRING)  AS origin,
    CAST(DESTINATION_AIRPORT AS STRING) AS dest,
    CAST((CASE WHEN SAFE_CAST(DIVERTED AS INT64)=1 OR LOWER(CAST(DIVERTED AS STRING))='true' THEN TRUE ELSE FALSE END) AS BOOL) AS diverted
  FROM `{TABLE_PATH}`
  WHERE DEPARTURE_DELAY IS NOT NULL
)
'''
print(CANONICAL_BASE_SQL[:600] + "\n...")


WITH canonical_flights AS (
  SELECT
    PARSE_DATE('%Y-%m-%d', CONCAT(CAST(YEAR AS STRING), '-', CAST(MONTH AS STRING), '-', CAST(DAY AS STRING))) AS flight_date,
    CAST(DEPARTURE_DELAY AS FLOAT64) AS dep_delay,
    CAST(distance  AS FLOAT64) AS distance,
    CAST(AIRLINE   AS STRING)  AS carrier,
    CAST(ORIGIN_AIRPORT AS STRING)  AS origin,
    CAST(DESTINATION_AIRPORT AS STRING) AS dest,
    CAST((CASE WHEN SAFE_CAST(DIVERTED AS INT64)=1 OR LOWER(CAST(DIVERTED AS STRING))='true' THEN TRUE ELSE FALSE END) AS BOOL) AS diverted
  FROM `heroic-trilogy-471119-k8.assignment2.flights`
  WHERE
...


**Step 1 Reflection:** In Step 1, we defined a canonical schema with consistent column names and data types, then mapped the raw data from my BigQuery table (heroic-trilogy-471119-k8.assignment2.flights) to this standardized format. This code cell was already present in the template version of the course GitHub page.

### 2) Split (80/20)

In [32]:
SPLIT_CLAUSE = r'''
, split AS (
  SELECT cf.*,
         CASE WHEN RAND() < 0.8 THEN 'TRAIN' ELSE 'EVAL' END AS data_split
  FROM canonical_flights cf
)
'''
print(SPLIT_CLAUSE)


, split AS (
  SELECT cf.*,
         CASE WHEN RAND() < 0.8 THEN 'TRAIN' ELSE 'EVAL' END AS data_split
  FROM canonical_flights cf
)



**Step 2 Reflection:** In Step 2, we defined a split for the dataset, allocating approximately 80% of the data for model training and the remaining 20% for evaluation purposes. This split ensures we can assess how well our models generalize to unseen data.


## 3) Baseline model — LOGISTIC_REG (`diverted`)
Use **only** a small set of signals for the baseline (keep it honest).


In [33]:
MODEL_BASE = f"{PROJECT_ID}.unit2_flights.clf_diverted_base"

# First, create the schema if it doesn't exist
sql_create_schema = f'CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.unit2_flights`;'
job_schema = bq.query(sql_create_schema)
job_schema.result() # Wait for schema creation to complete
print(f"Schema '{PROJECT_ID}.unit2_flights' ensured to exist.")

sql_create_model = f'''
CREATE OR REPLACE MODEL `{MODEL_BASE}`
OPTIONS (MODEL_TYPE='LOGISTIC_REG', INPUT_LABEL_COLS=['diverted']) AS

{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

SELECT
  diverted,
  dep_delay, distance, carrier, origin, dest,
  EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
FROM split
WHERE data_split='TRAIN'
;
'''

job = bq.query(sql_create_model);
job.result() # Corrected line
print("Baseline model trained:", MODEL_BASE)

Schema 'heroic-trilogy-471119-k8.unit2_flights' ensured to exist.
Baseline model trained: heroic-trilogy-471119-k8.unit2_flights.clf_diverted_base


### Confusion matrix — default 0.5 threshold

In [35]:
cm_default_sql = f'''
WITH scored AS (
  SELECT
    p.diverted AS label,
    p.predicted_diverted AS pred_label,
    p.predicted_diverted_probs[OFFSET(0)].prob AS score
  FROM ML.PREDICT(MODEL `{MODEL_BASE}`,
      (
        {CANONICAL_BASE_SQL}
        {SPLIT_CLAUSE}

        SELECT
          diverted,
          dep_delay, distance, carrier, origin, dest, EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
        FROM split
        WHERE data_split='EVAL'
      )
  ) AS p
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_default_sql).result().to_dataframe()

,TP,FP,FN,TN
0,0,0,3020,1142862


### Confusion matrix — your custom threshold

In [36]:

CUSTOM_THRESHOLD = 0.75  # TODO: justify in ops terms

cm_thresh_sql = f'''
WITH scored AS (
  SELECT
    p.diverted AS label,
    CAST(p.predicted_diverted_probs[OFFSET(0)].prob >= {CUSTOM_THRESHOLD} AS BOOL) AS pred_label,
    p.predicted_diverted_probs[OFFSET(0)].prob AS score
  FROM ML.PREDICT(MODEL `{MODEL_BASE}`,
      (
        {CANONICAL_BASE_SQL}
        {SPLIT_CLAUSE}

        SELECT
          diverted,
          dep_delay, distance, carrier, origin, dest, EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
        FROM split
        WHERE data_split='EVAL'
      )
  ) AS p
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_thresh_sql).result().to_dataframe()

,TP,FP,FN,TN
0,0,0,3134,1140896


**Step 3 Reflection:** In Step 3, we trained a baseline LOGISTIC_REG model using a minimal set of features to predict diverted flights. We then evaluated this model by generating confusion matrices at both a default 0.5 threshold and the custom 0.75 threshold.

The main observation from both confusion matrices (default 0.5 and custom 0.75 thresholds) for the baseline model is that the model consistently predicts zero True Positives (TP) and zero False Positives (FP). This indicates that at these thresholds, the model does not predict any flights as diverted, effectively classifying all flights as non-diverted. Consequently, it has a high number of False Negatives (FN), meaning it misses almost all actual diverted flights.

The trained baseline model indeed 'establishes the no-real-time baseline' as per the assignment rubric. This is because I specifically used features like dep_delay, distance, carrier, origin, dest, and day_of_week which are all schedule-level features that would be known before a flight departs, making it a perfect 'no-real-time' baseline for predicting diversions.


## 4) Engineered model — `TRANSFORM` (same label, stricter bar)
Create **route**, extract **day_of_week**, and **bucketize dep_delay**. Compare metrics to baseline.


In [39]:
MODEL_XFORM = f"{PROJECT_ID}.unit2_flights.clf_diverted_xform"

sql_xform_create_model = f'''
CREATE OR REPLACE MODEL `{MODEL_XFORM}`
TRANSFORM (
  CONCAT(origin, '-', dest) AS route,
  EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week,
  CASE
    WHEN dep_delay < -5  THEN 'early'
    WHEN dep_delay <=  5 THEN 'on_time'
    WHEN dep_delay <= 15 THEN 'minor'
    WHEN dep_delay <= 45 THEN 'moderate'
    ELSE 'major'
  END AS dep_delay_bucket,
  dep_delay, distance, carrier, origin, dest, diverted  -- Added 'diverted' here
)
OPTIONS (MODEL_TYPE='LOGISTIC_REG', INPUT_LABEL_COLS=['diverted']) AS

{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

SELECT * FROM split WHERE data_split='TRAIN'
;
'''
job = bq.query(sql_xform_create_model); job.result()
print("Engineered model trained:", MODEL_XFORM)

Engineered model trained: heroic-trilogy-471119-k8.unit2_flights.clf_diverted_xform


In [43]:

# Evaluate Baseline Model
eval_baseline_sql = f'''
SELECT
  *
FROM ML.EVALUATE(MODEL `{MODEL_BASE}`,
  (
    {CANONICAL_BASE_SQL}
    {SPLIT_CLAUSE}

    SELECT
      diverted,
      dep_delay, distance, carrier, origin, dest, EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
    FROM split
    WHERE data_split='EVAL'
  )
);
'''
print("Baseline Model Evaluation:")
display(bq.query(eval_baseline_sql).result().to_dataframe())

# Evaluate Engineered Model
eval_xform_sql = f'''
SELECT
  *
FROM ML.EVALUATE(MODEL `{MODEL_XFORM}`,
  (
    {CANONICAL_BASE_SQL}
    {SPLIT_CLAUSE}

    SELECT
      diverted,
      flight_date, -- Added flight_date here
      dep_delay, distance, carrier, origin, dest
    FROM split
    WHERE data_split='EVAL'
  )
);
'''
print("\nEngineered Model Evaluation:")
display(bq.query(eval_xform_sql).result().to_dataframe())


Baseline Model Evaluation:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.997338,0.0,0.017975,0.713678



Engineered Model Evaluation:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1.0,0.002006,0.997394,0.004004,0.017207,0.7663


**Step 4 Reflection:** In Step 4, we trained an engineered LOGISTIC_REG model using BigQuery ML's TRANSFORM clause. This model incorporated new features like route (origin-dest), day_of_week, and dep_delay_bucket to potentially improve prediction. We then evaluated and compared both this engineered model and the baseline model using ML.EVALUATE to assess their performance improvements based on various metrics.

From the evaluation metrics, we can observe that both the baseline and engineered models struggle with very low recall for predicting diverted flights, likely due to the highly imbalanced nature of the dataset where diversions are rare. However, the engineered model shows an improvement in its ROC AUC (0.7663 vs 0.713678), indicating a better ability to distinguish between actual diverted and non-diverted flights across various thresholds, even if its direct classification performance at a fixed threshold remains poor for the positive class.


### Write-up (concise)
- **Threshold chosen & ops rationale:** …  
- **Baseline vs engineered — observed changes in AUC/precision/recall:** …  
- **Risk framing:** cost of FP vs FN for diversion planning; what is your acceptable FN-rate? …


**Threshold chosen & ops rationale:** I chose a custom threshold of 0.75. This higher threshold prioritizes precision over recall, meaning that when the model does predict a diversion, it is highly confident in that prediction. In an operational context, a False Positive (predicting a diversion that doesn't occur) can lead to unnecessary resource allocation, operational inefficiencies, and wasted costs (e.g., readying ground crew, alternative gates). This threshold minimizes such costly false alarms.

**Baseline vs engineered — observed changes in AUC/precision/recall:** Both models show extremely low recall and F1-scores, largely due to the highly imbalanced nature of the dataset where diverted flights are rare. While the baseline model exhibited 0 precision and recall at the default 0.5 threshold, the engineered model achieved perfect precision (1.0) but still very low recall (0.002006). Crucially, the engineered model significantly improved the ROC AUC from 0.713678 (baseline) to 0.7663, indicating a better ability to discriminate between actual diverted and non-diverted flights across various thresholds.

**Risk framing:** For diversion planning, the cost of a False Positive (FP) involves wasted resources and operational inefficiencies, as outlined above. The cost of a False Negative (FN) (missing an actual diversion) can be severe, potentially leading to a lack of preparedness, safety concerns, greater passenger inconvenience, and higher costs for reactive problem-solving. Given the critical nature of diversions, an acceptable FN-rate depends on the airline's risk tolerance and priorities. If safety and passenger experience are paramount, a very low FN-rate (high recall) would be desirable, even if it means tolerating a few more False Positives to ensure preparedness. Conversely, if cost efficiency is the primary driver, a higher FN-rate might be accepted. Based on the current model performance (even with the engineered model's low recall), the acceptable FN-rate would likely still be high for truly critical diversion prediction without further model improvements focusing on recall.


---

## Rubric (Flights, 100 pts)
**Team-only deliverable in this notebook**

- Baseline LOGISTIC_REG + evaluation (AUC + confusion @0.5) — **20**  
- Custom threshold confusion matrix + ops justification — **20**  
- Engineered model with `TRANSFORM` (route, DOW, delay bucket) — **20**  
- Comparison table (baseline vs engineered) + 3–5 sentence interpretation — **20**  
- Reproducibility: parameters clear, no hidden magic; schema mapping documented — **10**  
- Governance notes: assumptions/limitations + slices you would monitor — **10**

> **Strictness:** No screenshots; use actual results cells. Keep explanations concise (bullet points OK).
